In [4]:
import hnswlib
import numpy as np
import struct
import os
import random
from sklearn.decomposition import PCA

In [5]:
def ed(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

def generate_matrix(lowdim, D):
    return np.random.normal(size=(lowdim, D))

def get_compact_sign_matrix(sgn_m, dt):
    return np.packbits(sgn_m, axis=1, bitorder='little').view(dt)

def read_hnsw_index(filepath, D):
    '''
    Read hnsw index from binary file
    '''
    index = hnswlib.Index(space='l2', dim=D)
    index.load_index(filepath)
    return index

def read_fvecs(filename, c_contiguous=True):
    fv = np.fromfile(filename, dtype=np.float32)
    if fv.size == 0:
        return np.zeros((0, 0))
    dim = fv.view(np.int32)[0]
    assert dim > 0
    fv = fv.reshape(-1, 1 + dim)
    if not all(fv.view(np.int32)[:, 0] == dim):
        raise IOError("Non-uniform vector sizes in " + filename)
    fv = fv[:, 1:]
    if c_contiguous:
        fv = fv.copy()
    return fv
    
def to_ivecs(filename: str, array: np.ndarray):
    print(f"Writing File - {filename}")
    topk = (np.int32)(array.shape[1])
    array = array.astype(np.int32)
    topk_array = np.array([topk] * len(array), dtype='int32')
    file_out = np.column_stack((topk_array, array.view('int32')))
    file_out.tofile(filename)
    

def to_fvecs(filename, data):
    print(f"Writing File - {filename}")
    with open(filename, 'wb') as fp:
        for y in data:
            d = struct.pack('I', len(y))
            fp.write(d)
            for x in y:
                a = struct.pack('f', x)
                fp.write(a)

# return internal ids           
def get_neighbors_with_internal_id(data_level_0, internal_id, size_data_per_element):
    start = int(internal_id * size_data_per_element / 4)
    cnt = data_level_0[start]
    neighbors = []
    for i in range(cnt):
        neighbors.append(data_level_0[start + i + 1])
    return neighbors

# return internal ids
def get_neighbors_with_external_label(data_level_0, external_label, size_data_per_element, label2id):
    internal_id = label2id[external_label]
    return get_neighbors_with_internal_id(data_level_0, internal_id, size_data_per_element)

In [6]:

source = './'
dataset = 'gist'
ef = 500
M = 16


In [7]:
path = os.path.join(source, dataset)
index_path = os.path.join(path, f'DWT_{dataset}_ef{ef}_M{M}.index')

data_path = os.path.join(path, f'{dataset}_base.fvecs')
# read data vectors
print(f"Reading {dataset} from {data_path}.")
X = read_fvecs(data_path)

Reading gist from ./gist/gist_base.fvecs.


In [8]:
index = read_hnsw_index(index_path, 961)
cnt = index.get_current_count()
print(f"totally {cnt} items in index")
capacity = index.get_max_elements()
print(f"capaciuty = {capacity}")
id_list = index.get_ids_list()
min_id = min(id_list)
max_id = max(id_list)
assert len(id_list) == cnt

totally 1000000 items in index
capaciuty = 1000000


In [9]:
ids = np.array([0,1,2])
ann_data = index.getAnnData()
data_level_0 = ann_data['data_level0']
size_data_per_element = ann_data['size_data_per_element']
offset_data = ann_data['offset_data']
internal_ids = ann_data['label_lookup_internal']
external_ids = ann_data['label_lookup_external']
id2label = {}
label2id = {}
for i in range(len(internal_ids)):
    id2label[internal_ids[i]] = external_ids[i]
    label2id[external_ids[i]] = internal_ids[i]

In [10]:
X.shape

(1000000, 960)

In [14]:
# np.sum(X*X, axis=1).shape

### Finger

- $|q_{proj} - d_{proj}|^2$
    - $=(t-b)^2|c|^2$
        - $|c|^2$ **(pre-calculated)**
        - $b$ **(pre-calculated)**
        - $t=\frac{q^Tc}{|c|^2}$
            - $q^Tc = \frac{|q|^2 + |c|^2 + |q-c|^2}{2}$
            - |q|^2 **online**, |q-c|^2 **(pre-dist)online**

- $|d_{res}|^2$ **pre-calculated**

- $|q_{res}|^2$
    - $=|q|^2 - |q_{proj}|^2$
        - $ |q_{proj}|^2 = t^2|c|^2$

- $q_{res}^Td_{res}$
    - $|q_{res}|*|d_{res}|*cos(q_{res},d_{res})$

In [15]:
# lsh_dim = 64
# num_int_lsh = int(lsh_dim / 32)

# # data_size = X.shape[0]
# data_size = 10000
# total_cnt = 0
# startIdx = np.zeros(data_size, dtype=np.int32)

# for i in range(data_size):
#     # print(i)
#     startIdx[i] = total_cnt
#     neighbors = get_neighbors_with_external_label(data_level_0, i, size_data_per_element, label2id)
#     total_cnt += len(neighbors)

# # |c|^2
# c_2 = np.zeros(data_size)

# # b, d_res
# edge_info_float = np.zeros((total_cnt, 2), dtype=np.float32)

# # sgn array of lsh(d_res)
# edge_info_int = np.zeros((total_cnt, num_int_lsh), dtype=np.int32)

# cur_idx = 0
# # for cur_label in range(data_size):
# # with None:
# cur_label = 76
# # print(cur_label)

# cur_vec = X[cur_label]
# cur_c_2 = cur_vec @ cur_vec
# c_2[cur_label] = cur_c_2
# neighbors = get_neighbors_with_external_label(data_level_0, cur_label, size_data_per_element, label2id)
# # assert(startIdx[cur_label] == cur_idx)
# for nei in neighbors:
#     b = 0
#     d_res = 0
#     if cur_c_2 != 0:
#         nei_vec = X[id2label[nei]]

#         b = (cur_vec @ nei_vec) / cur_c_2

#         d_2 = nei_vec @ nei_vec
#         d_proj_2 = b * b * cur_c_2
#         d_res = np.sqrt(d_2 - d_proj_2)
#         if d_2 < d_proj_2:
#             print(d_2, d_proj_2)
        
#     edge_info_float[cur_idx] = [b, d_res]
#     # print([b, d_res])
#     cur_idx += 1

In [11]:
lsh_dim = 64
# binary_view_type = 'uint16' # lsh:16->uint16; lsh:32->uint32; lsh:64->uint32;
# num_int_lsh = int(lsh_dim / 32)

P = generate_matrix(X.shape[1], lsh_dim)
# P.shape

In [12]:
data_size = X.shape[0]
# data_size = 100
total_cnt = 0
startIdx = np.zeros(data_size, dtype=np.int32)



for i in range(data_size):
    # print(i)
    startIdx[i] = total_cnt
    neighbors = get_neighbors_with_external_label(data_level_0, i, size_data_per_element, label2id)

    total_cnt += len(neighbors)

In [52]:
# d_res_vecs = np.zeros()

d_res_vecs = []
# for i in np.random.choice(data_size, 100000):
for i in range(data_size):
    neighbors = get_neighbors_with_external_label(data_level_0, i, size_data_per_element, label2id)
    nei_labels = [id2label[nei] for nei in neighbors]
    nei_vecs = X[nei_labels]
    c_2 = X[i] @ X[i]
    if c_2 != 0:
        nei_proj_vecs = np.outer((nei_vecs @ X[i]) / c_2, X[i])
        nei_res_vecs = nei_vecs - nei_proj_vecs
        d_res_vecs += list(nei_res_vecs)
# generate P with eigenvector of d res vectors

In [53]:
np.array(d_res_vecs).shape

(1594959, 960)

In [54]:
# pca = PCA(n_components=lsh_dim)
pca = PCA(n_components=64)
pca.fit(d_res_vecs)

PCA(n_components=64)

In [58]:
pca.components_.shape

(64, 960)

In [59]:
P = pca.components_.T

In [63]:
# |c|^2
# c_2 = np.sum(X * X, axis = 1)
c_2s = np.zeros(data_size, dtype=np.float32)
node_info_float = np.zeros((data_size, lsh_dim), dtype=np.float32)

# b, d_res
edge_info_float = np.zeros((total_cnt, 2), dtype=np.float32)

# sgn array of lsh(d_res)
# edge_info_uint = np.zeros((total_cnt, num_int_lsh), dtype=np.uint32)
edge_info_uint = np.zeros((total_cnt, lsh_dim), dtype=np.uint32)


cur_idx = 0
for cur_label in range(data_size):
    assert(cur_idx == startIdx[cur_label])
    cur_vec = X[cur_label]
    cur_c_2 = cur_vec @ cur_vec
    c_2s[cur_label] = cur_c_2
    neighbors = get_neighbors_with_external_label(data_level_0, cur_label, size_data_per_element, label2id)
    num_nei = len(neighbors)

    c_P = cur_vec @ P
    node_info_float[cur_label, :] = c_P
    # print(cur_label)

    if cur_c_2 == 0:
        edge_info_float[cur_idx: cur_idx+num_nei, :] = np.zeros((num_nei, 2))
    else:
        nei_labels = [id2label[nei] for nei in neighbors]
        nei_vecs = X[nei_labels]
        # print(nei_vecs.shape)

        bs = nei_vecs @ cur_vec / cur_c_2
        
        # cur_vec 

        d_2s = np.sum(nei_vecs * nei_vecs, axis=1, dtype=np.float32)
        d_proj_2s = bs * bs * cur_c_2
        d_res_2s = d_2s - d_proj_2s
        if np.any(d_res_2s < 0):
            d_res_2s[d_res_2s < 0] = 0
        d_ress = np.sqrt(d_res_2s)

        edge_info_float[cur_idx: cur_idx+num_nei, :] = np.array([bs, d_ress]).T

        
        # use little bit order to compact sign array. (might need changing the order according to OS)
        # two examples assuming 4 bits per compact num: 
        #   1, 0, 1, 0 -> 0b0101;
        #   1, 0, 1, 0 | 1, 1, 1, 0 -> 0b0101, 0b0111
        
        d_res_vecs_dot_P = nei_vecs @ P - np.outer(bs, cur_vec) @ P
        edge_info_uint[cur_idx: cur_idx+num_nei, :] = (np.sign(d_res_vecs_dot_P) > 0)
        
        # edge_info_uint[cur_idx: cur_idx+num_nei, :] = get_compact_sign_matrix(np.sign(d_res_vecs_dot_P)>0, binary_view_type)
        
    cur_idx += num_nei

In [55]:
def get_dist_ground_truth(q_vec, c_vec, d_vec):
    q_proj_vec = (q_vec @ c_vec) / (c_vec @ c_vec) * c_vec
    d_proj_vec = (d_vec @ c_vec) / (c_vec @ c_vec) * c_vec
    proj_delta = q_proj_vec - d_proj_vec
    part1 = proj_delta @ proj_delta

    d_res_vec = d_vec - d_proj_vec
    part2 = d_res_vec @ d_res_vec

    q_res_vec = q_vec - q_proj_vec
    part3 = q_res_vec @ q_res_vec

    part4 = -2 * d_res_vec @ q_res_vec
    part4_ = -2 * np.sqrt(part2) * np.sqrt(part3) * np.cos(np.sum(np.sign(q_res_vec @ P) != np.sign(d_res_vec @ P)) / P.shape[1] * np.pi)
    
    tmp = [part1, part2, part3, part4_]
    return tmp, np.sum(tmp)


In [21]:
def finger_dist(t, b, d_res, c_2, q_res, q_res_2, sgn_q_res_P, sgn_d_res_P):
    part1 = (t-b) * (t-b) * c_2

    part2 = d_res * d_res

    part3 = q_res_2

    cos = np.cos(np.sum(sgn_q_res_P != sgn_d_res_P) / len(sgn_d_res_P) * np.pi)
    part4 = -2 * q_res * d_res * cos

    tmp = [part1, part2, part3, part4]
    return tmp, np.sum(tmp)

In [64]:
# TODO implement distance calculation to verify the correctness

for _ in range(1000):
    # print(_)
    q_label = random.randint(0, data_size)
    c_label = random.randint(0, data_size)
    old_dist = ed(X[q_label], X[c_label])
    old_dist = old_dist * old_dist

    q_vec = X[q_label]
    c_vec = X[c_label]
    
    neighbors = get_neighbors_with_external_label(data_level_0, c_label, size_data_per_element, label2id)

    c_2 = c_2s[c_label]
    c_P = node_info_float[c_label]
    c_start = startIdx[c_label]

    q_2 = q_vec @ q_vec

    t = (q_2 + c_2 - old_dist) / 2 / c_2
    # tmp_t = q_vec @ c_vec / c_2
    # print(tmp_t - t)

    q_res_2 = q_2 - t * t * c_2

    ################
    # tmp_q_proj_vec = (q_vec @ c_vec) / (c_vec @ c_vec) * c_vec
    # tmp_q_res_vec = q_vec - tmp_q_proj_vec
    # tmp_q_res = np.sqrt(tmp_q_res_vec@tmp_q_res_vec)
    ################

    q_res = np.sqrt([q_res_2,0][int(q_res_2<0)])

    # print(tmp_q_res - q_res)

    q_res_P = q_vec @ P - t * c_P

    # print(np.sum(q_res_P - tmp_q_res_vec @ P))

    sgn_q_res_P = (np.sign(q_res_P) > 0)

    for d_offset, nei_id in enumerate(neighbors):
        nei_label = id2label[nei_id]
        d_vec = X[nei_label]

        dist1 = ed(q_vec, X[nei_label])
        dist1 = dist1 * dist1

        b = edge_info_float[c_start + d_offset][0]

        d_res = edge_info_float[c_start + d_offset][1]
        sgn_d_res_P = edge_info_uint[c_start + d_offset]

        part_result, dist2 = finger_dist(t, b, d_res, c_2, q_res, q_res_2, sgn_q_res_P, sgn_d_res_P)

        part_ground_result, ground_dist2 = get_dist_ground_truth(q_vec, c_vec, d_vec)
        # print(tmp_q_res*tmp_q_res - part_ground_result[3])
        print(np.array(part_result) - np.array(part_ground_result))
        # print(ground_dist2 - dist1)

        # print(abs(dist1 - abs(dist1 - dist2)) / dist1)
        # if abs(dist1 - dist2) > 1e-6:
        #     print("error when checking property!")
        #     break


[ 4.85638644e-07 -5.96046448e-07  3.48699368e-07 -2.60763855e-09]
[ 5.97985303e-07  1.31130219e-06  3.48699368e-07 -5.98332784e-07]
[ 4.94389071e-07 -2.38418579e-07  3.48699368e-07 -1.12112092e-07]
[ 6.07296289e-07 -2.38418579e-07  3.48699368e-07 -1.32876974e-07]
[-5.94084226e-07  9.53674316e-07  3.48699368e-07 -3.84394179e-07]
[-5.44503336e-08  4.76837158e-07  3.48699368e-07 -5.56877019e-07]
[-4.86473760e-07  1.07288361e-06  3.48699368e-07 -3.92486728e-07]
[-1.05649408e-06  3.09944153e-06  3.48699368e-07 -1.32999481e-06]
[-2.34735073e-07  3.57627869e-07  6.32225658e-08 -1.79525437e-07]
[ 1.81032900e-09  3.57627869e-07  6.32225658e-08 -2.04776948e-07]
[-1.86871938e-08 -5.96046448e-07  6.32225658e-08  3.28489030e-07]
[ 9.97845861e-09  7.15255737e-07  6.32225658e-08 -4.04422418e-07]
[-7.34528682e-09 -3.57627869e-07  6.32225658e-08  7.64286039e-08]
[-6.93504081e-07  1.66893005e-06  6.32225658e-08 -1.11923648e-06]
[-1.63621304e-08  7.15255737e-07  6.32225658e-08 -3.53446501e-07]
[-2.435118

In [23]:
print(np.packbits([1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],bitorder='little').view(np.uint8),0b10100000_00000000)
print(np.outer(np.array([1,2,3]), np.array([1,3,5])))

[5 0] 40960
[[ 1  3  5]
 [ 2  6 10]
 [ 3  9 15]]


In [46]:
startIdx[:100]

array([   0,    9,   38,   67,   87,  118,  135,  165,  197,  221,  229,
        261,  286,  315,  345,  359,  376,  401,  414,  446,  468,  500,
        526,  558,  584,  616,  633,  665,  697,  725,  736,  754,  769,
        799,  826,  855,  881,  903,  925,  926,  955,  987, 1001, 1033,
       1046, 1074, 1095, 1124, 1133, 1139, 1166, 1177, 1194, 1214, 1233,
       1261, 1271, 1302, 1331, 1363, 1371, 1400, 1425, 1454, 1477, 1500,
       1516, 1542, 1573, 1598, 1615, 1647, 1670, 1697, 1706, 1729, 1742,
       1774, 1800, 1832, 1864, 1885, 1896, 1928, 1941, 1959, 1985, 2017,
       2036, 2065, 2079, 2107, 2133, 2153, 2184, 2216, 2246, 2267, 2288,
       2303], dtype=int32)

In [61]:
# TODO implement distance calculation to verify the correctness

for _ in range(1000):
    # print(_)
    q_label = random.randint(0, data_size)
    c_label = random.randint(0, data_size)
    old_dist = ed(X[q_label], X[c_label])
    old_dist = old_dist * old_dist

    q_vec = X[q_label]
    c_vec = X[c_label]
    
    neighbors = get_neighbors_with_external_label(data_level_0, c_label, size_data_per_element, label2id)

    for d_offset, nei_id in enumerate(neighbors):
        nei_label = id2label[nei_id]
        d_vec = X[nei_label]

        dist1 = ed(q_vec, X[nei_label])
        dist1 = dist1 * dist1

        part_ground_result, ground_dist2 = get_dist_ground_truth(q_vec, c_vec, d_vec)
        print(abs(dist1 - abs(dist1 - ground_dist2)) / dist1)


0.8562690783994833
0.9372796304602753
0.8856180346161812
0.6926067320769632
0.9027101839053658
0.9195719830066462
0.9986956486192129
0.8560006995794335
0.7730030791381175
0.9868315256900464
0.8133940187416965
0.8274186849780504
0.9836049939349346
0.9272348120752216
0.9740640447457416
0.9715787907569295
0.9783140004386106
0.9916100315372438
0.9213767084709084
0.9726090537109413
0.9703268938601124
0.9439549474255171
0.905823504203215
0.9503600208580419
0.9549401308318598
0.9498054341581554
0.8890488929179754
0.9243860156785632
0.9370856864761778
0.8614741236212015
0.9410378314857573
0.9252941046920501
0.8973541369144296
0.9593988809058961
0.876832708422976
0.9649785664084432
0.9767041619305482
0.9253359652723152
0.9605811918479968
0.9573607832585723
0.9081005752963189
0.941029169149928
0.9259362639263767
0.9390472399976376
0.9113523286291524
0.996338721982849
0.9974337868508487
0.8567237756343882
0.8468531024286866
0.7389909455458824
0.8497977279725535
0.9524656100443981
0.87484296093678

In [62]:
P.shape

(960, 64)

In [69]:
# c_2 = np.sum(X * X, axis = 1)
# c_2s = np.zeros(data_size)
# node_info_float = np.zeros((data_size, lsh_dim), dtype=np.float32)

# b, d_res
# edge_info_float = np.zeros((total_cnt, 2), dtype=np.float32)

# sgn array of lsh(d_res)
# edge_info_uint = np.zeros((total_cnt, lsh_dim), dtype=np.uint32)

projection_path = os.path.join(path, f'FINGER_{dataset}_LSH_{lsh_dim}.fvecs')
b_dres_path = os.path.join(path, f'FINGER_{dataset}_b_dres.fvecs')
sgn_dres_P_path = os.path.join(path, f'FINGER_{dataset}_sgn_dres_P.ivecs')
c_2_path = os.path.join(path, f'FINGER_{dataset}_c_2.fvecs')
c_P_path = os.path.join(path, f'FINGER_{dataset}_c_P.fvecs')
start_idx_path = os.path.join(path, f'FINGER_{dataset}_start_idx.ivecs')

to_fvecs(projection_path, P)
to_fvecs(b_dres_path, edge_info_float)
to_ivecs(sgn_dres_P_path, edge_info_uint)
to_fvecs(c_2_path, c_2s.astype(np.float32).reshape(data_size,1))
to_fvecs(c_P_path, node_info_float)
to_ivecs(start_idx_path, startIdx.reshape(data_size,1))


Writing File - ./gist/FINGER_gist_c_2.fvecs
Writing File - ./gist/FINGER_gist_start_idx.fvecs


In [68]:
print(c_2s)

[7.72810125 6.68669462 9.28546524 ... 7.19309521 4.86128998 3.77380252]
